# 1. Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 1.1. Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at the [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf)'s *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original (correct) word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 1.1.1. Downloading the dataset

In [ ]:
import urllib.request
data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 1.1.2. Implementation of search

In [ ]:
import re
import nltk

def build_inverted_index_orig_forms(documents):
    inverted_index = {}
    
    for doc_id, doc in enumerate(documents):
        for word in set(nltk.word_tokenize(doc.lower())):
            if word.isalpha():
                if word not in inverted_index:
                    inverted_index[word] = [1, (doc_id, 1)]
                else:
                    inverted_index[word][0] += 1
                    inverted_index[word].append((doc_id, 1))
    
    return inverted_index

def build_k_gram_index(inverted_index, k):
    k_gram_index = {}
    
    for word in inverted_index.keys():
        padded_word = f"${word}$"
        for i in range(len(padded_word) - k + 1):
            k_gram = padded_word[i:i+k]
            if k_gram not in k_gram_index:
                k_gram_index[k_gram] = [word]
            else:
                k_gram_index[k_gram].append(word)
    
    return k_gram_index

def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    padded_wildcard = f"${wildcard}$"
    wildcard_k_grams = [padded_wildcard[i:i+3] for i in range(len(padded_wildcard) - 2)]
    candidate_sets = [set(k_gram_index[k_gram]) for k_gram in wildcard_k_grams if k_gram in k_gram_index]

    if not candidate_sets:
        return []

    common_words = set.intersection(*candidate_sets)
    regex = re.compile("^" + wildcard.replace("*", ".*") + "$")
    matching_words = [word for word in common_words if regex.match(word)]
    
    return matching_words

def search_wildcard(wildcard, k_gram_index, index, docs):
    wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index)
    regex = re.compile(r'\b(?:' + '|'.join(wildcard_options) + r')\b', flags=re.IGNORECASE)
    results = [doc for doc in docs if regex.search(doc)]
    
    return results

In [ ]:
# split_str_3gram = lambda s, k=3: list(map(''.join, zip(*[iter(s)]*k)))

n = 3
your_string=  "$wild$"
[your_string[i:i+n] for i in range(0, len(your_string)-2)]

# split_str_3gram("$wild$")

['$wi', 'wil', 'ild', 'ld$']

In [ ]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)
k_gram_index['$th']

['the',
 'their',
 'than',
 'there',
 'that',
 'these',
 'they',
 'them',
 'three',
 'things',
 'thigh',
 'thought',
 'this',
 'themselves',
 'thirsty',
 'those']

### 1.1.3. Tests

In [ ]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)
assert("red" not in wildcard_options) 

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['reduced', 'recorded', 'received']
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
102. More than 50% of the people in the world have never made or received a telephone call.
134. A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you'll feel thirsty. If it's reduced by 10%, you'll die.


## 1.2. Handling typos

### 1.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [ ]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz

--2023-05-08 20:09:16--  https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
Resolving github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)... 3.5.25.191, 52.217.192.49, 52.217.47.124, ...
Connecting to github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)|3.5.25.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43769081 (42M) [application/x-gzip]
Saving to: ‘github-typo-corpus.v1.0.0.jsonl.gz’

github-typo-corpus. 100%[===================>]  41.74M  71.3MB/s    in 0.6s    

2023-05-08 20:09:17 (71.3 MB/s) - ‘github-typo-corpus.v1.0.0.jsonl.gz’ saved [43769081/43769081]



In [ ]:
# !tar -xvzf github-typo-corpus.v1.0.0.jsonl.gz

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
!gunzip github-typo-corpus.v1.0.0.jsonl.gz


In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [ ]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### 1.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [ ]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [ ]:
from collections import Counter
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [ ]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 1.2.1. Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [ ]:

def jaccard_index(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def fix_typo_kgram(word, k_gram_index, k=3, top_n=5):
    padded_word = f"${word}$"
    word_k_grams = [padded_word[i:i+k] for i in range(len(padded_word) - k + 1)]
    word_k_grams_set = set(word_k_grams)
    
    candidate_words = set()
    for k_gram in word_k_grams:
        if k_gram in k_gram_index:
            candidate_words.update(k_gram_index[k_gram])
    
    jaccard_scores = [(candidate, jaccard_index(word_k_grams_set, set([f"${candidate}$"[i:i+k] for i in range(len(candidate) - k + 2)]))) for candidate in candidate_words]
    jaccard_scores.sort(key=lambda x: -x[1])
    
    best_matches = [word_score_pair[0] for word_score_pair in jaccard_scores[:top_n]]
    
    return best_matches


### 1.2.2. Tests

In [ ]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'eno', 'enought', 'endogenous', 'enoent']


## 1.3. [Extra tasks, for fun]

### 1.3.1. QWERTY - Editorial distance

Write the code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [ ]:
qwerty = [
    "1234567890",
    "qwertyuiop",
    "asdfghjkl",
    "zxcvbnm"
]

keyboard_positions = {}
for row, keys in enumerate(qwerty):
    for col, key in enumerate(keys):
        keyboard_positions[key] = (row, col)


In [ ]:
def replace_weight(let1, let2):
    qwerty_rows = ["1234567890", "qwertyuiop", "asdfghjkl", "zxcvbnm"]
    row_multiplier = [0.5, 0.7]
    
    for i, row in enumerate(qwerty_rows):
        if let1 in row and let2 in row:
            if abs(row.index(let1) - row.index(let2)) == 1:
                return row_multiplier[i % 2]
        elif (let1 in qwerty_rows[1] and let2 in qwerty_rows[2]) or (let1 in qwerty_rows[2] and let2 in qwerty_rows[1]):
            let1_index = row.index(let1) if let1 in row else None
            let2_index = row.index(let2) if let2 in row else None
            if let1_index is not None and let2_index is not None:
                if abs(let1_index - let2_index) <= 1:
                    return 0.7
    return 1.0


def qwerty_edit_dist(s1, s2):
    len1 = len(s1)
    len2 = len(s2)
    d = [[0.0 for _ in range(len2 + 1)] for _ in range(len1 + 1)]

    for i in range(len1 + 1):
        d[i][0] = i

    for j in range(len2 + 1):
        d[0][j] = j

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = replace_weight(s1[i - 1], s2[j - 1]) if s1[i - 1] != s2[j - 1] else 0.0
            d[i][j] = min(
                d[i - 1][j] + 1,  # deletion
                d[i][j - 1] + 1,  # insertion
                d[i - 1][j - 1] + cost  # substitution
            )
            if i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]:
                d[i][j] = min(d[i][j], d[i - 2][j - 2] + 1)  # transposition

    return d[len1][len2]

# tests
assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivue", "figure") == 0.7, "Edit distance is computed incorrectly"


### 1.3.2. Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [ ]:
# import re
from collections import Counter

def words(text):
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words("your large corpus of text goes here"))

def P(word, N=sum(WORDS.values())): 
  res = WORDS[word] / N
  return 0.01 if res==0 else res

def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    return set(w for w in words if w in WORDS)

def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts    = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def fix_typo_qwerty_norvig(word):
    return min(candidates(word), key=lambda w: qwerty_edit_dist(word, w) / P(w))


In [ ]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

ZeroDivisionError: ignored

### 12.3.3. Estimate quality of functions

In [ ]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, wk = fix_typo_qwerty_norvig(word), \
                     fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")